# Тестовое задание MacBuildServer

Вывести отчёт по заявкам за произвольный период в днях (от 1 дня и более)

Данные нужно забрать с помощью Google Sheets API из документа https://docs.google.com/spreadsheets/d/1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ

Формат отчёта:

- Dimensions
    * канал привлечения заявки (d_utm_source)
    * клуб (d_club)
    * менеджер (d_manager)
- Metrics
    * количество заявок
    * количество мусорных заявок (на основании заявки не создан клиент)
    * количество новых заявок (не было заявок и покупок от этого клиента раньше)
    * количество покупателей (кто купил в течение недели после заявки)
    * количество новых покупателей (кто купил в течение недели после заявки и не покупал раньше)
    * доход от покупок новых покупателей

Итоговый отчёт нужно вывести в свой документ Google Sheets:
* Для обработки данных написать код на Python
* Данные выгружать из Google Sheets и загружать в свой документ через Google Sheets API
* Для итогового отчёта использовать pivot table Google Sheets
* (Бонусное задание, необязательно) вывести итоговый отчёт в виде pivot table DataStudio

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pandas as pd
import numpy as np
import sqlite3

In [2]:
print(gspread.__version__)
print(pd.__version__)
print(np.__version__)
print(sqlite3.sqlite_version)

3.7.0
1.2.3
1.20.1
3.35.5


### Загрузка данных

In [3]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('named-storm-315718-9f45fd3ec26b.json', scope) 

gc = gspread.authorize(credentials)
sh = gc.open_by_key('1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ')

In [4]:
worksheet = sh.get_worksheet(0)

data = worksheet.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)
print(df.head())

  Данные находятся в соседних вкладках                       
0                                                            
1        Легенда по названиям колонок:                       
2                        m_real_amount          сумма покупки
3                               client           клиент в CRM
4                                 lead  входящая заявка в CRM


In [5]:
worksheet = sh.get_worksheet(1)
data = worksheet.get_all_values()
headers = data.pop(0)
transactions = pd.DataFrame(data, columns=headers)
print(transactions.columns)

worksheet = sh.get_worksheet(2)
data = worksheet.get_all_values()
headers = data.pop(0)
clients = pd.DataFrame(data, columns=headers)
print(clients.columns)

worksheet = sh.get_worksheet(3)
data = worksheet.get_all_values()
headers = data.pop(0)
managers = pd.DataFrame(data, columns=headers)
print(managers.columns)

worksheet = sh.get_worksheet(4)
data = worksheet.get_all_values()
headers = data.pop(0)
leads = pd.DataFrame(data, columns=headers).replace({'d_utm_source': {'': 'unknown_source'}})
print(leads.columns)

Index(['transaction_id', 'created_at', 'm_real_amount', 'l_client_id'], dtype='object')
Index(['client_id', 'created_at', 'l_manager_id'], dtype='object')
Index(['manager_id', 'd_manager', 'd_club'], dtype='object')
Index(['lead_id', 'created_at', 'd_utm_medium', 'd_utm_source', 'l_manager_id',
       'l_client_id'],
      dtype='object')


In [6]:
report_period = 10

start_point = (pd.to_datetime(max(leads.created_at)) +\
               pd.DateOffset(-report_period)).strftime('%Y-%m-%d %H:%M:%S')

В таблице managers не хватает записей

In [7]:
managers.manager_id.unique().size >= leads.l_manager_id.unique().size

False

In [8]:
unknown_manager_id = leads.l_manager_id.unique()[~np.isin(leads.l_manager_id.unique(), managers.manager_id)]

managers_expanded = managers.append(
    pd.DataFrame(unknown_manager_id, columns=['manager_id'])\
    .assign(d_manager=unknown_manager_id, d_club=[np.nan for _ in unknown_manager_id])).fillna('unknown club')

In [9]:
managers_expanded.manager_id.unique().size >= leads.l_manager_id.unique().size

True

### Количество заявок

In [10]:
leads_summary = pd.pivot_table(
    leads.merge(managers_expanded, left_on='l_manager_id', right_on='manager_id', how='inner')
    .query('created_at >= @start_point'), 
    values='lead_id', index=['d_manager', 'd_club'], columns=['d_utm_source'], aggfunc='count', fill_value=0)

leads_summary.sum().sum()

620

### Количество мусорных заявок (на основании заявки не создан клиент)

In [11]:
trash_leads = pd.pivot_table(
    leads.merge(clients, left_on='l_client_id', right_on='client_id', how='left')\
    .query('client_id.isnull() & created_at_x >= @start_point')\
    .merge(managers_expanded, left_on='l_manager_id_x', right_on='manager_id', how='inner'),
    values='lead_id', index=['d_manager', 'd_club'], columns=['d_utm_source'], aggfunc='count', fill_value=0)

trash_leads.sum().sum()

154

### Количество новых заявок (не было заявок и покупок от этого клиента раньше)

In [12]:
cnx = sqlite3.connect(':memory:')
transactions.to_sql(name='transactions_db', con=cnx)
leads.to_sql(name='leads_db', con=cnx)

In [13]:
query = '''
WITH 
union_tab (operation_id, operation_type, created_at, l_client_id) 
AS (
    SELECT transaction_id, "transaction", created_at, l_client_id
    FROM transactions_db
    UNION ALL
    SELECT lead_id, "lead", created_at, l_client_id
    FROM leads_db
    )
SELECT l_client_id
FROM 
    (
    SELECT *, ROW_NUMBER() OVER(PARTITION BY l_client_id ORDER BY created_at) rank
    FROM union_tab
    ) AS t
WHERE operation_type = "lead" AND rank = 1

'''

Список клиентов, у которых покупки начанаются не раньше первой заявки

In [14]:
client_id = pd.read_sql(query, cnx).l_client_id
client_id.size

1790

Список клиентов, у которых не было раньше заявок и покупок 

In [15]:
new_client_id = leads.query('l_client_id in @client_id')\
.groupby('l_client_id').count().query('lead_id == 1').index

new_client_id.size

1568

In [16]:
new_leads = pd.pivot_table(
    leads.merge(managers_expanded, left_on='l_manager_id', right_on='manager_id', how='inner')\
    .query('l_client_id in @new_client_id & created_at >= @start_point'),
    values='lead_id', index=['d_manager', 'd_club'], columns=['d_utm_source'], aggfunc='count', fill_value=0)\

new_leads.sum().sum()

282

### Количество покупателей (кто купил в течение недели после заявки)

In [17]:
query = '''
WITH 
union_tab (operation_id, operation_type, created_at, l_client_id) 
AS (
    SELECT transaction_id, "transaction", created_at, l_client_id
    FROM transactions_db
    UNION ALL
    SELECT lead_id, "lead", created_at, l_client_id
    FROM leads_db
    ),
delay_tab (operation_id, operation_type, created_at, l_client_id, operation_lag, delay)
AS (
    SELECT *, 
        LAG(operation_type) OVER(PARTITION BY l_client_id ORDER BY created_at),
        JULIANDAY(created_at) - 
        JULIANDAY(LAG(created_at) OVER(PARTITION BY l_client_id ORDER BY created_at)) 
    FROM union_tab
    )
SELECT DISTINCT l_client_id
FROM delay_tab 
WHERE operation_type = "transaction" AND operation_lag = "lead" AND delay <= 7

'''

In [18]:
customer_id = pd.read_sql(query, cnx).l_client_id
customer_id.size

400

In [19]:
customers = pd.pivot_table(
    leads.merge(managers_expanded, left_on='l_manager_id', right_on='manager_id', how='inner')\
    .query('l_client_id in @customer_id & created_at >= @start_point'),
    values='l_client_id', index=['d_manager', 'd_club'], columns=['d_utm_source'], aggfunc='count', fill_value=0)

customers.sum().sum()

82

### Количество новых покупателей (кто купил в течение недели после заявки и не покупал раньше)

In [20]:
new_customer_id = customer_id[customer_id.isin(new_client_id)]
new_customer_id.size

266

In [21]:
new_customers = pd.pivot_table(
    leads.merge(managers_expanded, left_on='l_manager_id', right_on='manager_id', how='inner')\
    .query('l_client_id in @new_customer_id & created_at >= @start_point'),
    values='l_client_id', index=['d_manager', 'd_club'], columns=['d_utm_source'], aggfunc='count', fill_value=0)

new_customers.sum().sum()

42

### Доход от покупок новых покупателей

In [22]:
fresh_income = pd.pivot_table(
    leads.merge(managers_expanded, left_on='l_manager_id', right_on='manager_id', how='inner')\
    .merge(transactions, on='l_client_id', how='inner')\
    .query('l_client_id in @new_customer_id & created_at_x >= @start_point')\
    .assign(m_real_amount = lambda x: x.m_real_amount.astype(int)),
    values='m_real_amount', index=['d_manager', 'd_club'], columns=['d_utm_source'], aggfunc=np.sum, fill_value=0)

fresh_income.sum().sum()

1262771

### Сводная таблица

In [23]:
summary = pd.melt(leads_summary.reset_index(), id_vars=['d_manager', 'd_club'], value_vars=leads_summary.columns,
                  value_name='leads')\
.merge(pd.melt(trash_leads.reset_index(), id_vars=['d_manager', 'd_club'], value_vars=trash_leads.columns, 
               value_name='trash_leads'),
       on=['d_manager', 'd_club', 'd_utm_source'], how='left')\
.merge(pd.melt(new_leads.reset_index(), id_vars=['d_manager', 'd_club'], value_vars=trash_leads.columns, 
               value_name='new_leads'),
       on=['d_manager', 'd_club', 'd_utm_source'], how='left')\
.merge(pd.melt(customers.reset_index(), id_vars=['d_manager', 'd_club'], value_vars=customers.columns, 
               value_name='customers'),
       on=['d_manager', 'd_club', 'd_utm_source'], how='left')\
.merge(pd.melt(new_customers.reset_index(), id_vars=['d_manager', 'd_club'], value_vars=new_customers.columns, 
               value_name='new_customers'),
       on=['d_manager', 'd_club', 'd_utm_source'], how='left')\
.merge(pd.melt(fresh_income.reset_index(), id_vars=['d_manager', 'd_club'], value_vars=fresh_income.columns, 
               value_name='fresh_income'),
       on=['d_manager', 'd_club', 'd_utm_source'], how='left')\
.fillna(0).set_index(['d_manager', 'd_club', 'd_utm_source']).astype('int')

### Выгрузка данных

In [24]:
sh = gc.open_by_key('11g_0iRx1qWnPwJ_uBJtrsSETTsVA31G9yPnnfJnfOF0')

set_with_dataframe(sh.get_worksheet(0), leads_summary, include_index=True)
set_with_dataframe(sh.get_worksheet(1), trash_leads, include_index=True)
set_with_dataframe(sh.get_worksheet(2), new_leads, include_index=True)
set_with_dataframe(sh.get_worksheet(3), customers, include_index=True)
set_with_dataframe(sh.get_worksheet(4), new_customers, include_index=True)
set_with_dataframe(sh.get_worksheet(5), fresh_income, include_index=True)
set_with_dataframe(sh.get_worksheet(6), summary, include_index=True)

### Бонусное задание

https://datastudio.google.com/reporting/29742ffb-f66a-4a32-8bb1-8801dfbaeedd